In [1]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
from torch import optim
from sklearn.metrics import classification_report

In [2]:
transform=transforms.Compose([
    transforms.ToTensor(),#tranforms to the tensor
    transforms.Resize((256,256),),
    transforms.Normalize((0.5,),(0.5,))#just copied
])
root=r"C:\Users\saipr\OneDrive\Documents\PotatoPlants"
dataset=datasets.ImageFolder(root=root,transform=transform)#it will divide classes w.r.t to no.of folders inside it

In [3]:
train_size=int(0.9*(len(dataset)))
test_size=len(dataset)-train_size
train_data,test_data=random_split(dataset,[train_size,test_size])
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)#train_data(that we are passing) should be tensor(my understanding)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)


In [4]:
class_names=dataset.classes#gives the  classes names
class_idx=dataset.class_to_idx

In [5]:
class CNN(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.network=nn.Sequential(#intially I wrote self as nn
            nn.Conv2d(in_channels=3,out_channels=4,kernel_size=3,stride=1,padding=1),#these in channels are rbg colors of a colored photo
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=4,out_channels=8,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),     
            nn.Linear(32*8*8,64),
            nn.ReLU(),
            nn.Linear(64,output_size)            
        )
    def forward(self,x):
        return self.network(x)

In [6]:
model=CNN(input_size=256*256,output_size=len(dataset.classes))
criterion=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.001)

In [15]:
num_epochs=20
for epoch in range(num_epochs):
 for input,output in train_loader:
    pred_output=model(input)
    loss=criterion(pred_output,output)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
 print(f'Loss in Epoch [{epoch+1}/{num_epochs}]:{loss} ')
 if loss<0.01:
   break


tensor([[ 2.0099e+01,  2.5131e+00, -3.2393e+00],
        [ 1.3269e+01,  1.5584e+00, -9.2570e-01],
        [ 2.2251e+01,  1.8963e+00, -5.7367e+00],
        [-8.0928e+00,  9.2573e-01,  2.8265e+00],
        [-1.0618e+00,  5.3887e+00, -1.3377e+00],
        [-2.4152e+00,  3.7720e+00, -1.5180e+00],
        [-2.3434e+00,  1.9780e+00,  1.0973e+00],
        [ 1.3307e+01,  2.8680e+00, -2.1945e+00],
        [ 2.0819e+00,  6.2412e+00, -3.8312e+00],
        [-7.1577e+00,  2.9826e+00,  9.7147e-01],
        [ 2.5001e+01,  3.6326e+00, -6.4738e+00],
        [ 2.2006e+01, -4.4656e-01, -2.1523e+00],
        [ 2.0185e+01,  1.9608e+00, -5.0078e+00],
        [-4.0261e+00,  2.3056e+00,  1.4720e+00],
        [-7.1258e+00,  4.0281e+00, -1.7406e+00],
        [ 1.5580e+01,  4.1083e+00, -5.3511e+00],
        [ 1.6559e+01,  2.1167e+00, -3.8589e+00],
        [ 1.6325e+01, -7.7928e-01,  3.6716e-01],
        [-2.3592e+00,  1.7490e-01,  4.5339e+00],
        [ 1.2963e+01,  2.0656e+00,  2.6038e-01],
        [ 1.8020e+01

In [8]:
def ClassificationReport(model,loader):
    y_test,y_pred=[],[]
    model.eval()
    with torch.no_grad():
        for input,output in loader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
    model.train()
    print(classification_report(y_test,y_pred))
    
    for i in range(3):
        tc=sum(1 for y,x in zip(y_pred,y_test) if (y==i)&(x==i))
        ac=sum(1 for y in y_test if y==i)
        print(f'{list(class_idx.keys())[i]}: [{tc}/{ac}]')


In [9]:
print('classification report of training phase:')
ClassificationReport(model,train_loader)
print('Classification Report of testing phase:')
ClassificationReport(model,test_loader)

classification report of training phase:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       902
           1       0.99      0.95      0.97       903
           2       0.82      0.88      0.85       131

    accuracy                           0.97      1936
   macro avg       0.93      0.94      0.93      1936
weighted avg       0.97      0.97      0.97      1936

Potato___Early_blight: [902/902]
Potato___Late_blight: [858/903]
Potato___healthy: [115/131]
Classification Report of testing phase:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        98
           1       0.94      0.94      0.94        97
           2       0.88      0.71      0.79        21

    accuracy                           0.94       216
   macro avg       0.93      0.88      0.90       216
weighted avg       0.94      0.94      0.94       216

Potato___Early_blight: [98/98]
Potato___Late_blight: [91/97]
Pot

In [10]:
def accuracy(model,dataloader,classes):
    model.eval()
    with torch.no_grad():
        y_test,y_pred=[],[]
        for input,output in dataloader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
        #classes=dataloader.classes
        cor,tot=[],[]
        for i in range(len(classes)):
            cor.append(sum(1 for j,k in zip(y_test,y_pred) if j==i and k==i))
            tot.append(sum (1 for j in y_test if j==i))
        print(f'accuracy report of {dataloader}:')
        for i,cl in enumerate(classes):
            print(f'{cl}:[{cor[i]}/{tot[i]}]')
        print(classification_report(y_test,y_pred))
    model.train()
print(accuracy(model,test_loader,class_names))
print(accuracy(model,train_loader,class_names))


accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x000001E8995F3B30>:
Potato___Early_blight:[98/98]
Potato___Late_blight:[91/97]
Potato___healthy:[15/21]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        98
           1       0.94      0.94      0.94        97
           2       0.88      0.71      0.79        21

    accuracy                           0.94       216
   macro avg       0.93      0.88      0.90       216
weighted avg       0.94      0.94      0.94       216

None
accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x000001E8994774A0>:
Potato___Early_blight:[902/902]
Potato___Late_blight:[858/903]
Potato___healthy:[115/131]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       902
           1       0.99      0.95      0.97       903
           2       0.82      0.88      0.85       131

    accuracy                           0.9

In [11]:
pvp_root=r"C:\Users\saipr\Downloads\archive\PlantVillage\PlantVillage"
pvp_dataset=datasets.ImageFolder(root=pvp_root,transform=transform)
pvp_dataloader=DataLoader(pvp_dataset,batch_size=64,shuffle=True)
print(accuracy(model,pvp_dataloader,pvp_dataset.classes))

accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x000001E8991CA750>:
Potato___Early_blight:[1000/1000]
Potato___Late_blight:[949/1000]
Potato___healthy:[130/152]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1000
           1       0.98      0.95      0.96      1000
           2       0.83      0.86      0.84       152

    accuracy                           0.97      2152
   macro avg       0.93      0.93      0.93      2152
weighted avg       0.97      0.97      0.97      2152

None
